In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
import tensorflow
import cv2
import numpy as np
import matplotlib.pyplot as plt
import keras.layers as layers
from keras.models import Model
from keras.preprocessing import image

from tensorflow.keras.regularizers import l2

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os
import cv2
from google.colab.patches import cv2_imshow
from tensorflow.keras.layers import Conv2D

from keras import backend as K
from keras.datasets import mnist
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from sklearn.utils import shuffle

import numpy as np
import os
from PIL import Image
import scipy.io as sio

from keras.initializers import glorot_uniform

#from keras.layers.normalization import BatchNormalization
#from tensorflow.keras.models import Sequential

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from keras.models import Sequential
#from keras.optimizers import adam, SGD
from keras.callbacks import ModelCheckpoint
from keras.constraints import maxnorm
from keras.models import load_model
from keras.layers import GlobalAveragePooling2D, Lambda, Conv2D, MaxPooling2D, Dropout, Dense, Flatten, Activation
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
trainpath="/content/gdrive/MyDrive/adv_orig_Dataset/mat_adv_orig_Dataset/mat_orig_cw_adv_Dataset/Training"
testpath="/content/gdrive/MyDrive/adv_orig_Dataset/mat_adv_orig_Dataset/mat_orig_cw_adv_Dataset/Testing"

In [ ]:
class_type= os.listdir(trainpath)

In [ ]:
class_type

['orig', 'adv']

In [ ]:
def load_dataset(path,class_type):
  labels = []
  mats = []
  for item in class_type:
    filenames = os.listdir(path + '/' + item)
    for f in filenames:
      mat_contents = sio.loadmat(path + '/' + item +'/' + f)
      if 'orig' in mat_contents:
        img=np.array((mat_contents['orig']*255).astype('uint8'))
        labels.append(0)
      else:
        img=np.array((mat_contents['adv']*255).astype('uint8'))
        labels.append(1)
      resized_image = cv2.resize(img, (227, 227))   
      mats.append(np.array(resized_image/255))  

  mats = np.array(mats)
  labels = np.array(labels) 
  mats,labels = shuffle(mats, labels, random_state = 45)
  return mats,labels     



In [ ]:
train_x,train_y=load_dataset(trainpath,class_type)

In [ ]:
test_x,test_y=load_dataset(testpath,class_type)

In [ ]:
train_y

array([1, 0, 1, ..., 0, 0, 1])

In [ ]:
img_rows = 224
img_cols = 224
img_chan = 3
input_shape=(img_rows, img_cols, img_chan)

In [ ]:
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten,\
 Conv2D, MaxPooling2D,BatchNormalization




# (3) Create a sequential model
model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(227,227,3), kernel_size=(11,11),\
 strides=(4,4), padding='same'))
model.add(Activation('relu'))
# Pooling 
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same'))
# Batch Normalisation before passing it to the next layer
model.add(BatchNormalization())

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding='same'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same'))
# Batch Normalisation
model.add(BatchNormalization())

# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same'))
# Batch Normalisation
model.add(BatchNormalization())

# Passing it to a dense layer
model.add(Flatten())
# 1st Dense Layer
model.add(Dense(4096, input_shape=(227*227*3,)))
model.add(Activation('relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.2))
# Batch Normalisation
model.add(BatchNormalization())

# 2nd Dense Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.2))
# Batch Normalisation
model.add(BatchNormalization())

# 3rd Dense Layer
model.add(Dense(1000))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.2))
# Batch Normalisation
model.add(BatchNormalization())

# Output Layer
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 57, 57, 96)        34944     
_________________________________________________________________
activation (Activation)      (None, 57, 57, 96)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 29, 29, 96)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 29, 29, 96)        384       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 256)       614656    
_________________________________________________________________
activation_1 (Activation)    (None, 29, 29, 256)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 256)       0

In [ ]:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=45,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)  # randomly flip images

datagen.fit(train_x)

In [ ]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
#model.fit(train_x, train_y, batch_size=32,epochs=400, validation_data=(test_x,test_y),shuffle=True)

In [ ]:
augmented_model_details = model.fit_generator(datagen.flow(train_x, train_y, batch_size = 32),
                     # number of samples per gradient update
                    epochs = 200, # number of iterations
                    validation_data=(test_x,test_y),
                    verbose=1)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/200
44/44 [==============================] - 20s 417ms/step - loss: 0.8410 - accuracy: 0.4946 - val_loss: 1.1307 - val_accuracy: 0.2382
Epoch 2/200
44/44 [==============================] - 18s 399ms/step - loss: 0.7136 - accuracy: 0.5018 - val_loss: 0.8500 - val_accuracy: 0.3936
Epoch 3/200
44/44 [==============================] - 18s 398ms/step - loss: 0.7099 - accuracy: 0.4837 - val_loss: 1.0234 - val_accuracy: 0.1943
Epoch 4/200
44/44 [==============================] - 18s 399ms/step - loss: 0.7012 - accuracy: 0.4938 - val_loss: 0.8181 - val_accuracy: 0.3530
Epoch 5/200
44/44 [==============================] - 18s 399ms/step - loss: 0.7026 - accuracy: 0.5011 - val_loss: 0.7565 - val_accuracy: 0.4020
Epoch 6/200
44/44 [==============================] - 18s 401ms/step - loss: 0.6978 - accuracy: 0.5250 - val_loss: 0.8827 - val_accuracy: 0.2787
Epoch 7/200
44/44 [==============================] - 18s 406ms/step - loss: 0.7050 - accuracy: 0.4946 - val_loss: 0.7165 - val_accuracy:

In [ ]:
loss, accuracy = model.evaluate(test_x, test_y)
print('Loss = ' + str(loss))
print('Test Accuracy = ' + str(accuracy))